In [1]:
import pandas as pd

In [26]:
df = pd.read_csv('data/EV_data.csv')
df.head()

,read_date,interval_1,interval_2,interval_3,interval_4,interval_5,interval_6,interval_7,interval_8,interval_9,...,interval_41,interval_42,interval_43,interval_44,interval_45,interval_46,interval_47,interval_48,id,label
0,3/1/2021,0.0625,0.0500,0.0687,0.0750,0.0687,0.0500,0.0625,0.0687,0.0687,...,0.0812,0.0687,0.0687,0.0562,0.0562,0.0687,0.0687,0.0625,1,1
1,3/2/2021,0.0625,0.0500,0.0687,0.0625,0.0625,0.0562,0.0562,0.0625,0.0687,...,0.1375,0.0750,0.0687,0.0625,0.0625,0.0562,0.0625,0.0625,1,1
2,3/3/2021,0.0625,0.0687,0.0500,0.0562,0.0687,0.0625,0.0625,0.0625,0.0500,...,0.1875,0.1062,0.0750,0.0687,0.0687,0.0625,0.0562,0.0562,1,1
3,3/4/2021,0.0625,0.0625,0.0687,0.0500,0.0562,0.0625,0.0625,0.0625,0.0562,...,0.1125,0.0875,0.0687,0.0625,0.0562,0.0625,0.0625,0.0687,1,1
4,3/5/2021,0.0625,0.0625,0.0500,0.0625,0.0687,0.0625,0.0687,0.0562,0.0500,...,0.0812,0.0562,0.0687,0.0625,0.0625,0.0625,0.0562,0.0500,1,1


In [27]:
X = df.drop(['id','read_date','label'],axis=1)
y = df['label']
print(X.head())
print(y.head())

   interval_1  interval_2  interval_3  interval_4  interval_5  interval_6  \
0      0.0625      0.0500      0.0687      0.0750      0.0687      0.0500   
1      0.0625      0.0500      0.0687      0.0625      0.0625      0.0562   
2      0.0625      0.0687      0.0500      0.0562      0.0687      0.0625   
3      0.0625      0.0625      0.0687      0.0500      0.0562      0.0625   
4      0.0625      0.0625      0.0500      0.0625      0.0687      0.0625   

   interval_7  interval_8  interval_9  interval_10  ...  interval_39  \
0      0.0625      0.0687      0.0687       0.0500  ...       0.1187   
1      0.0562      0.0625      0.0687       0.0625  ...       0.1187   
2      0.0625      0.0625      0.0500       0.0625  ...       0.1812   
3      0.0625      0.0625      0.0562       0.0500  ...       0.1000   
4      0.0687      0.0562      0.0500       0.0625  ...       0.1062   

   interval_40  interval_41  interval_42  interval_43  interval_44  \
0       0.1062       0.0812       

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB

In [3]:
df.isna().sum()

read_date      0
interval_1     0
interval_2     0
interval_3     0
interval_4     0
interval_5     0
interval_6     0
interval_7     0
interval_8     0
interval_9     0
interval_10    0
interval_11    0
interval_12    0
interval_13    0
interval_14    0
interval_15    0
interval_16    0
interval_17    0
interval_18    0
interval_19    0
interval_20    0
interval_21    0
interval_22    0
interval_23    0
interval_24    0
interval_25    0
interval_26    0
interval_27    0
interval_28    0
interval_29    0
interval_30    0
interval_31    0
interval_32    0
interval_33    0
interval_34    0
interval_35    0
interval_36    0
interval_37    0
interval_38    0
interval_39    0
interval_40    0
interval_41    0
interval_42    0
interval_43    0
interval_44    0
interval_45    0
interval_46    0
interval_47    0
interval_48    0
id             0
label          0
dtype: int64

In [16]:
# Classifiers
classifiers = {
    "LogisticRegression" : LogisticRegression(random_state=0),
    "KNN" : KNeighborsClassifier(),
    
    "RandomForest" : RandomForestClassifier(random_state=0),
    #"XGBoost" : XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss'), # XGBoost takes too long
    "LGBM" : LGBMClassifier(random_state=0),
    
    "NaiveBayes": GaussianNB()
}

In [17]:
# Grids for grid search
LR_grid = {'penalty': ['l1','l2'],
           'C': [0.25, 0.5, 0.75, 1, 1.25, 1.5],
           'max_iter': [50, 100, 150]}

KNN_grid = {'n_neighbors': [3, 5, 7, 9],
            'p': [1, 2]}



RF_grid = {'n_estimators': [50, 100, 150, 200, 250, 300],
        'max_depth': [4, 6, 8, 10, 12]}

boosted_grid = {'n_estimators': [50, 100, 150, 200],
        'max_depth': [4, 8, 12],
        'learning_rate': [0.05, 0.1, 0.15]}

NB_grid={'var_smoothing': [1e-10, 1e-9, 1e-8, 1e-7]}

# Dictionary of all grids
grid = {
    "LogisticRegression" : LR_grid,
    "KNN" : KNN_grid,
    
    "RandomForest" : RF_grid,
    "XGBoost" : boosted_grid,
    "LGBM" : boosted_grid,
    
    "NaiveBayes": NB_grid
}

In [28]:
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid = train_test_split(X,y,stratify=y,train_size=0.8,test_size=0.2,random_state=0)

In [10]:
import time
from sklearn.model_selection import GridSearchCV
import  numpy as np

In [29]:
i=0
clf_best_params=classifiers.copy()
valid_scores=pd.DataFrame({'Classifer':classifiers.keys(), 'Validation accuracy': np.zeros(len(classifiers)), 'Training time': np.zeros(len(classifiers))})
for key, classifier in classifiers.items():
    start = time.time()
    clf = GridSearchCV(estimator=classifier, param_grid=grid[key], n_jobs=-1, cv=None)

    # Train and score
    clf.fit(X_train, y_train)
    valid_scores.iloc[i,1]=clf.score(X_valid, y_valid)

    # Save trained model
    clf_best_params[key]=clf.best_params_
    
    # Print iteration and training time
    stop = time.time()
    valid_scores.iloc[i,2]=np.round((stop - start)/60, 2)
    
    print('Model:', key)
    print('Training time (mins):', valid_scores.iloc[i,2])
    print('')
    i+=1

/Users/bijubiju/Desktop/ml projects/current/space_titanic/venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
90 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/bijubiju/Desktop/ml projects/current/space_titanic/venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/bijubiju/Desktop/ml projects/current/space_titanic/venv/lib/python3.10/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/bijubi

Model: LogisticRegression
Training time (mins): 0.01

Model: KNN
Training time (mins): 0.02

Model: RandomForest
Training time (mins): 0.67

[LightGBM] [Info] Number of positive: 1295, number of negative: 1295
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001709 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12240
[LightGBM] [Info] Number of data points in the train set: 2590, number of used features: 48
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 1295, number of negative: 1295
[LightGBM] [Info] Number of positive: 1295, number of negative: 1296
[LightGBM] [Info] Number of positive: 1295, number of negative: 1295
[

In [30]:
valid_scores

,Classifer,Validation accuracy,Training time
0,LogisticRegression,0.624691,0.01
1,KNN,0.848148,0.02
2,RandomForest,0.881481,0.67
3,LGBM,0.937037,3.16
4,NaiveBayes,0.625926,0.00


In [31]:
clf_best_params

{'LogisticRegression': {'C': 0.25, 'max_iter': 50, 'penalty': 'l2'},
 'KNN': {'n_neighbors': 3, 'p': 1},
 'RandomForest': {'max_depth': 12, 'n_estimators': 100},
 'LGBM': {'learning_rate': 0.15, 'max_depth': 8, 'n_estimators': 200},
 'NaiveBayes': {'var_smoothing': 1e-10}}